In [1]:
from PIL import Image
import requests

import faiss
from transformers import CLIPProcessor, CLIPModel

In [3]:
%load_ext autoreload
%autoreload 2
import embeddings

## Reference Lists

In [30]:

req = requests.get("https://raw.githubusercontent.com/CSAILVision/places365/master/categories_places365.txt")
places_list = [x.split(' ') for x in req.content.decode('utf-8').split('\n')]
places = ["_".join(x[0].split('/')[2:]).replace("_", " ") for x in places_list]
with open("places_365.txt", 'w') as f:
    f.write("\n".join(places))

In [113]:
oi = []
with open("2017_11/class-descriptions.csv") as f:
    for line in f:
        oi.append(line.strip())
        
with open('objects_open_images.txt', 'w') as f:
    f.write("\n".join([x.split(',')[-1] for x in oi]))

## Embeddings

In [4]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [5]:
image = Image.open('n02086240_7268.JPEG')

In [6]:
with open("places_365.txt") as f:
    places = f.read().split('\n')

In [7]:
inputs = processor(text=places, return_tensors="pt", padding=True)
places_embeddings = model.get_text_features(**inputs)

In [23]:
places_index = FaissIndex(embedding_size=768, faiss_index_location='faiss_indices/places.index', indexer=faiss.IndexFlatIP)
places_index.reset()

In [24]:
places_index.add(places_embeddings.detach().numpy(), places)

In [13]:
# inputs = tokenizer(places, padding=True, return_tensors="pt")
image_inputs = processor(images=image, return_tensors="pt", padding=True)
out = model.get_image_features(**image_inputs)

In [25]:
places_index.search(out.detach().numpy())

(array([[0.17297854, 0.15687369, 0.15423924, 0.15403895, 0.15325068,
         0.14620401, 0.1395231 , 0.13754925, 0.13722014, 0.13552673]],
       dtype=float32),
 array([[ 50, 346, 201, 173, 261, 303, 208, 155,  63, 221]]),
 ['beauty salon',
  'veterinarians office',
  'kennel outdoor',
  'hayfield',
  'pet shop',
  'shower',
  'laundromat',
  'galley',
  'bow window indoor',
  'manufactured home'])

In [21]:
with open("objects_open_images.txt") as f:
    objects = f.read().split('\n')